# Kaggle Agents

**Autonomous Agent System with Reinforcement Learning & Meta-Evaluation**

This notebook demonstrates the capabilities of the Kaggle Agents system developed for the Specialization Course in AI.

**Quick Start:**
1. Runtime > Change runtime type > GPU (Recommended)
2. Run all cells
3. Watch the agents solve competitions autonomously!

## 1. Environment Setup

Clone repository and install dependencies.

In [1]:
# Check GPU availability
!nvidia-smi || echo "No GPU available (CPU mode)"

Mon Nov 24 00:22:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Clone repository
!git clone https://github.com/gustavogomespl/kaggle-agents.git
%cd kaggle-agents

In [ ]:
# Install dependencies
!uv pip install --system -e .
!uv pip install --system langgraph langchain-openai pandas numpy scikit-learn matplotlib seaborn

print("✓ Installation complete!")

## 2. Configuration

Set up API keys and credentials.

In [4]:
import os
from pathlib import Path

# ============================================================
# 🔧 CONFIGURAÇÃO PRINCIPAL - EDITE AQUI
# ============================================================

# LLM Provider global: "openai" ou "anthropic"
LLM_PROVIDER = "openai"  # altere para "openai" se preferir GPT
LLM_MODEL = "claude-sonnet-4-5" if LLM_PROVIDER == "anthropic" else "gpt-5-mini-2025-08-07"
LLM_TEMPERATURE = 1
LLM_MAX_TOKENS = 30000

# Overrides por papel (opcional)
PLANNER_PROVIDER = "openai"
PLANNER_MODEL = "gpt-5-mini-2025-08-07"
DEVELOPER_PROVIDER = "openai"
DEVELOPER_MODEL = "gpt-5-mini-2025-08-07"
EVALUATOR_PROVIDER = "openai"
EVALUATOR_MODEL = "gpt-5-mini-2025-08-07"

# Option 1: Colab Secrets
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') or ""
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY') or ""
    KAGGLE_USERNAME = userdata.get('KAGGLE_USERNAME') or ""
    KAGGLE_KEY = userdata.get('KAGGLE_KEY') or ""
    print("✓ Loaded secrets from Colab")
except Exception:
    # Option 2: Manual
    from getpass import getpass
    print("⚠️ Colab secrets not found. Using manual input.")
    OPENAI_API_KEY = getpass("Enter OpenAI API Key (optional): ") or ""
    ANTHROPIC_API_KEY = getpass("Enter Anthropic API Key (optional): ") or ""
    KAGGLE_USERNAME = input("Enter Kaggle Username (optional): ") or ""
    KAGGLE_KEY = getpass("Enter Kaggle API Key (optional): ") or ""

env_content = f"""# Kaggle Agents Configuration

# LLM Settings
LLM_PROVIDER={LLM_PROVIDER}
LLM_MODEL={LLM_MODEL}
LLM_TEMPERATURE={LLM_TEMPERATURE}
LLM_MAX_TOKENS={LLM_MAX_TOKENS}

# Per-role overrides
PLANNER_PROVIDER={PLANNER_PROVIDER}
PLANNER_MODEL={PLANNER_MODEL}
DEVELOPER_PROVIDER={DEVELOPER_PROVIDER}
DEVELOPER_MODEL={DEVELOPER_MODEL}
EVALUATOR_PROVIDER={EVALUATOR_PROVIDER}
EVALUATOR_MODEL={EVALUATOR_MODEL}

# API Keys
OPENAI_API_KEY={OPENAI_API_KEY}
ANTHROPIC_API_KEY={ANTHROPIC_API_KEY}

# Kaggle Integration
KAGGLE_USERNAME={KAGGLE_USERNAME}
KAGGLE_KEY={KAGGLE_KEY}
KAGGLE_AUTO_SUBMIT=false

# Workflow Settings
MAX_ITERATIONS=3
TARGET_PERCENTILE=20.0

# Logging
LOG_LEVEL=INFO
LOG_DIR=./logs
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("\n✓ Configuration saved to .env")

# Setup Kaggle credentials
if KAGGLE_USERNAME and KAGGLE_KEY:
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(exist_ok=True)
    with open(kaggle_dir / 'kaggle.json', 'w') as f:
        f.write(f'{{"username":"{KAGGLE_USERNAME}","key":"{KAGGLE_KEY}"}}')
    !chmod 600 ~/.kaggle/kaggle.json
    print("✓ Kaggle credentials configured")
else:
    print("⚠️ Kaggle credentials not provided - auto-submission disabled")

✓ Loaded secrets from Colab

✓ Configuration saved to .env
✓ Kaggle credentials configured


## 3. Select Competition

Choose a competition to solve.

In [ ]:
# List active competitions
if KAGGLE_USERNAME and KAGGLE_KEY:
    print("Active Kaggle Competitions:\n")
    !kaggle competitions list --category=playground | head -n 10
else:
    print("Popular competitions:")
    print("  - titanic")
    print("  - house-prices-advanced-regression-techniques")
    print("  - spaceship-titanic")
    print("  - playground-series-s5e11")

COMPETITION = input("\nEnter competition name: ").strip() or "titanic"
MAX_ITERATIONS = int(input("Max iterations (default: 3): ").strip() or "3")

## 4. Run Autonomous Agents 

This starts the full LangGraph workflow:
1. **Search**: Finds SOTA solutions
2. **Plan**: Creates ablation strategy
3. **Develop**: Writes & fixes code
4. **Validate**: Checks for robustness
5. **Ensemble**: Combines best models
6. **Meta-Evaluate**: Optimizes with RL
7. **Explain**: Generates didactic report

In [ ]:
print("Configuring LangSmith Tracing...")

# Try to get from Colab secrets first
try:
    from google.colab import userdata
    langchain_api_key = userdata.get('LANGSMITH')
    langsmith_project = userdata.get('LANGSMITH_PROJECT')
    print("✓ Loaded secrets from Colab")
except:
    langchain_api_key = None

if not langchain_api_key:
    langchain_api_key = getpass("Enter LangSmith API Key (optional): ")

if langchain_api_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
    os.environ["LANGCHAIN_PROJECT"] = langsmith_project
    print("✓ LangSmith Tracing Enabled (Project: kaggle-agents-tcc)")

    # Update .env file if it exists
    if os.path.exists(".env"):
        with open(".env", "a") as f:
            f.write(f"\nLANGCHAIN_TRACING_V2=true\n")
            f.write(f"LANGCHAIN_ENDPOINT=https://api.smith.langchain.com\n")
            f.write(f"LANGCHAIN_API_KEY={langchain_api_key}\n")
            f.write(f"LANGCHAIN_PROJECT={langsmith_project}\n")
else:
    print("ℹ️ LangSmith Tracing Disabled")

Configuring LangSmith Tracing...
✓ Loaded secrets from Colab
✓ LangSmith Tracing Enabled (Project: kaggle-agents-tcc)


In [ ]:
from kaggle_agents.core import solve_competition
import time

print(f"\n{'='*60}")
print(f"Starting TCC Workflow: {COMPETITION}")
print(f"{'='*60}\n")

start_time = time.time()

try:
    results = solve_competition(
        competition_name=COMPETITION,
        competition_description=f"Solve {COMPETITION} competition",
        problem_type="binary_classification", # Will be auto-detected usually
        evaluation_metric="accuracy",
        max_iterations=MAX_ITERATIONS,
    )

    elapsed = time.time() - start_time
    print(f"\n✓ Workflow Complete in {elapsed:.1f}s")

except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:

# Submit to Kaggle
if KAGGLE_USERNAME and KAGGLE_KEY:
    work_dir = Path(f"/content/kaggle_competitions/competitions/{COMPETITION}")
    submission_file = work_dir / "submission.csv"

    if submission_file.exists():
        message = f"Kaggle Agents - Autonomous submission )"

        print(f"Submitting to {COMPETITION}...\n")
        !kaggle competitions submit -c {COMPETITION} -f {submission_file} -m "{message}"

        print(f"\n✓ Submission complete!")
        print(f"\nView results: https://www.kaggle.com/competitions/{COMPETITION}/submissions")
    else:
        print(f"❌ No submission file found: {submission_file}")
else:
    print("⚠️ Kaggle credentials not configured")
    print("\nManual submission:")
    print(f"1. Download submission.csv from working directory")
    print(f"2. Go to: https://www.kaggle.com/competitions/{COMPETITION}/submit")
    print(f"3. Upload the file")